In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import random
import cv2
import os
matplotlib.use('Agg')

In [2]:
def generate_images(history):
    for i in range(len(history)):
        data = history[i]

        # create discrete colormap
        cmap = colors.ListedColormap(['black',"lime",'blue',"red"])
        bounds = [0,0.9,1.5,2.5,3]
        norm = colors.BoundaryNorm(bounds, cmap.N)

        fig, ax = plt.subplots()
        ax.imshow(data, cmap=cmap, norm=norm)

        # draw gridlines
        ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
        ax.set_xticks(np.arange(-.5, 10, 1));
        ax.set_yticks(np.arange(-.5, 10, 1));
        ax.set_frame_on(False)
        plt.axis('off')
        plt.savefig(str(i)+".png",dpi=200, bbox_inches="tight")

def make_video(speed):
    print("Converting Frames to Video")
    img=[]
    i=0
    name=str(i)+'.png'
    while(os.path.exists(name)):
        img.append(cv2.imread(name))
        i+=1
        name=str(i)+'.png'

    count=i
    height,width,layers=img[1].shape

    video=cv2.VideoWriter('resultv.mp4',100,5*speed,(width,height))

    for j in range(0,count):
        video.write(img[j])

    cv2.destroyAllWindows()
    video.release()

    print("Done")

In [3]:
def get_head():
    size=len(square)
    for i in range(size):
        for j in range(size):
            if(square[i][j]==2):
                return i,j
            
def get_apple():
    size=len(square)
    for i in range(size):
        for j in range(size):
            if(square[i][j]==3):
                return i,j

def update(history):
    t=0
    size=len(square)
    
    snake_size=len(snake)
    
    for i in range(snake_size-1):
        for j in range(i+1,snake_size):
            if(snake[i]==snake[j]):
                print("hit yourself",end=" ")
                t=1
                
    x,y=snake[0]
    if(x<0) or (x>=size) or (y<0) or (y>=size):
        print("out the block",end=" ")
        t=1
        
    new_sq=[[0 for i in range(size)] for j in range(size)]
    for i in range(size):
        for j in range(size):
            if (i,j) in apple:
                square[i][j]=3
                new_sq[i][j]=3
            elif (i,j) in head:
                square[i][j]=2
                new_sq[i][j]=2
            elif (i,j) in snake:
                square[i][j]=1
                new_sq[i][j]=1
            else:
                square[i][j]=0
                new_sq[i][j]=0
    
    history+=[new_sq]
    
    if(t==1):
        history+=[-1]
    
    return history
    

def show():
    size=len(square)
    for i in range(size):
        for j in range(size):
            index=square[i][j]
            if(index==0):
                print("_",end="")
            elif(index==1):
                print("H",end="")
            elif(index==2):
                print("D",end="")
            else:
                print("O",end="")
        print("")
    print("")

def go_direction(snake,apple,ver,hor):

    (x,y)=snake[0]
    head=[(x+ver,y+hor)]
    if(head[0] in apple):
        snake=head+snake
        apple.remove(head[0])
        size=len(square)
        
        i=random.randint(0,size-1)
        j=random.randint(0,size-1)
        while((i,j) in snake):
            i=random.randint(0,size-1)
            j=random.randint(0,size-1)
        apple.append((i,j))
    else:
        snake=head+snake[:-1]
    
    return snake,head,apple

def update_direction(action,direction):
    direction=(direction+action)%4
    return direction
    
direction_dict={0:"d",1:"s",2:"a",3:"w"}

def direction_vector(move):
    move=direction_dict[move]
    
    if(move=="w"):
        (x,y)=(-1,0)
    elif(move=="s"):
        (x,y)=(1,0)
    elif(move=="a"):
        (x,y)=(0,-1)
    else:
        (x,y)=(0,1)
        
    return (x,y)

In [4]:
def relu(x):
    if x>0:
        return x
    else:
        return 0

def propagate(arr,w1,b1,w2,b2,w3,b3):
    size=len(arr)
    A=np.array(arr)
    A=np.reshape(A,size*size)
    L1=b1+np.matmul(w1,A)
    
    for i in range(len(L1)):
        L1[i]=relu(L1[i])
    
    L2=b2+np.matmul(w2,L1)
    
    for i in range(len(L2)):
        L2[i]=relu(L2[i])
        
    L3=b3+np.matmul(w3,L2)
    
    out=np.argmax(L3)-1
    
    return out

In [ ]:
W1=np.random.normal(0,1,(20,400))
B1=np.random.normal(0,1,20)
W2=np.random.normal(0,1,(20,20))
B2=np.random.normal(0,1,20)
W3=np.random.normal(0,1,(3,20))
B3=np.random.normal(0,1,3)  

In [14]:
highscore=0
count=0
all_scores=[]

for j in range(100000):
    
    count+=1
    print(count,end=" ")
    
    #Initalize Data
    
    n=20
    square=[[0 for i in range(n)] for j in range(n)]
    history=[]
    # 3 is apple
    # 2 is head
    # 1 is body

    actions=[]

    snake=[(10,10)]
    apple=[(6,5),(3,5),(5,9),(17,5),(13,11),(15,3),(3,18),(4,10),None]
    head=[snake[0]]
    direction=3
    score=1
    max_score=1
    # 0,1 is right
    # 0,-1 is left
    # 1,0 is down
    # -1,0 is up

    history=update(history)
    #show()

    time_to_evolve=0
    times=[]
    
    #Use Data and Run 
    
    for i in range(10000):
        total_time=i
        actions.append(direction_dict[direction])

        #move=random.randint(-1,1) #make algo for
        greed=random.random()
        if(greed<0.7):
            move=propagate(history[-1],W1,B1,W2,B2,W3,B3)
        else:
            move=random.randint(-1,1)
            
        direction=update_direction(move,direction)
        (x,y)=direction_vector(direction)

        snake,head,apple=go_direction(snake,apple,x,y)
        history=update(history)
        
        if(history[-1]==-1):
            #print("breaking cuz hit")
            break
            
        score=len(snake)

        if(max_score<score):
            times.append(time_to_evolve)
            max_score=score
            time_to_evolve=0
        elif(max_score==score):
            time_to_evolve+=1

        if(time_to_evolve>1000):
            print("breaking cuz timeout",end=" ")
            break

        #show()
    
    #print("outta loop")
    print(score,end=" ")
    all_scores.append(score)
    if(score>highscore) and (history[-1]!=-1):
        highscore=score
        
    if(score>=7):
        print(score,times,total_time)
        break

1 out the block 2 2 out the block 3 3 out the block 1 4 out the block 2 5 out the block 2 6 out the block 2 7 out the block 2 8 out the block 2 9 out the block 2 10 out the block 2 11 out the block 4 12 out the block 3 13 out the block 3 14 out the block 1 15 out the block 2 16 hit yourself 5 17 out the block 2 18 hit yourself 5 19 out the block 3 20 out the block 2 21 out the block 1 22 out the block 4 23 out the block 4 24 out the block 2 25 out the block 3 26 out the block 4 27 out the block 2 28 out the block 3 29 out the block 2 30 out the block 2 31 out the block 3 32 out the block 3 33 hit yourself 5 34 out the block 3 35 out the block 4 36 out the block 1 37 out the block 2 38 out the block 1 39 out the block 4 40 out the block 4 41 hit yourself 5 42 hit yourself 5 43 out the block 4 44 out the block 2 45 out the block 4 46 hit yourself 5 47 out the block 3 48 out the block 4 49 out the block 2 50 out the block 3 51 hit yourself 5 52 out the block 3 53 out the block 3 54 out th

 1 430 out the block 2 431 out the block 2 432 out the block 2 433 out the block 5 434 out the block 2 435 out the block 2 436 out the block 1 437 out the block 1 438 out the block 2 439 out the block 3 440 out the block 2 441 out the block 2 442 out the block 1 443 out the block 2 444 out the block 4 445 out the block 2 446 out the block 3 447 out the block 2 448 out the block 2 449 hit yourself 5 450 out the block 2 451 out the block 3 452 out the block 1 453 out the block 2 454 hit yourself 5 455 out the block 1 456 out the block 2 457 out the block 2 458 out the block 4 459 out the block 3 460 out the block 1 461 out the block 3 462 hit yourself 5 463 out the block 3 464 out the block 2 465 hit yourself 5 466 out the block 3 467 hit yourself 5 468 out the block 2 469 out the block 1 470 out the block 2 471 out the block 4 472 out the block 2 473 hit yourself 5 474 out the block 2 475 out the block 3 476 out the block 1 477 out the block 5 478 out the block 1 479 out the block 1 480

In [16]:
generate_images(history[:-1])

In [17]:
make_video(3)

Converting Frames to Video
Done
